# Compare the FY23 VMT-Mix with HPMS VMT-Mix

Created by: Apoorb  
Created on: July 20, 2023

Processing steps:

1. Percent of emissions from Running
2. Refactor running VMT to different vehicles
3. Recompute emissions
4. See the impact


In [88]:
from pathlib import Path
import pandas as pd
import geopandas as gpd
import numpy as np


In [89]:
POLS = [
    "Carbon Monoxide (CO)",
    "Oxides of Nitrogen (NOx)",
    "Volatile Organic Compounds",
    "CO2 Equivalent",
    "Primary Exhaust PM10  - Total",
    "Primary PM10 - Brakewear Particulate",
    "Primary PM10 - Tirewear Particulate",
    "Primary Exhaust PM2.5 - Total",
    "Primary PM2.5 - Brakewear Particulate",
    "Primary PM2.5 - Tirewear Particulate",
]


## Load Data


In [90]:
pa_data = Path(r"C:\Users\a-bibeka\PycharmProjects\FY23_VMT_Mix\data")
pa_emis = pa_data.joinpath("trends_2020_sutft_ei.csv")
pa_act = pa_data.joinpath("trends_2020_sutft_act.csv")
pa_vmix_fy23 = pa_data.joinpath("fy23_fin_vmtmix_13_21_032023.csv")
pa_vmix_hpms = pa_data.joinpath("PivotChartForVMTmixComparison.xlsx")
pa_rate_map = pa_data.joinpath("Rate_Categories.csv")
pa_county_shp = pa_data.joinpath("Texas_County_Boundaries", "County.shp")
pa_district_shp = pa_data.joinpath("TxDOT_Districts", "TxDOT_Districts.shp")
sutft_act = pd.read_csv(pa_act)
suft_emis = pd.read_csv(pa_emis)
vmtmix = pd.read_csv(pa_vmix_fy23)
rate_map = pd.read_csv(pa_rate_map)
gdf_county = gpd.read_file(pa_county_shp)
gdf_district = gpd.read_file(pa_district_shp)
df_district = gdf_district.filter(items=["DIST_NM", "DIST_NBR"])
district_map = gdf_county[["TXDOT_DIST", "FIPS_ST_CN", "CNTY_NM"]].astype({"TXDOT_DIST": int, "FIPS_ST_CN": int, "CNTY_NM": str})
x1 = pd.ExcelFile(pa_vmix_hpms)
vmtmix_hpms = x1.parse("DatasetForPivotChart", skiprows=4).loc[
    lambda df: (df.Year == 2020) & (df.DayType == "Wkd") & (df.TimeType == "day")
]
vmtmix_filt = vmtmix.loc[
    lambda df: (df.dowagg == "Wkd")
    & (df.yearID == 2020)
    & (df.tod == "day")
    & (df.mvs_rdtype_nm != "ALL")
].rename(
    columns={
        "sourceTypeID": "source_type_id",
        "fuelTypeID": "fuel_type_id",
        "vmt_mix": "vmt_mix_fy23",
    }
)


In [91]:
rename_sutft_emis = {
	'Period': 'period', 
	'Year': 'year_id', 
	'FIPS': 'fips', 
	'County': 'cnty_nm', 
	'Road Type ID': 'mvs_rdtype', 
	'Road Type': 'rdtype',
	'Source Type ID': 'source_type_id', 
	'Source Type': 'sut', 
	'Fuel Type ID': 'fuel_type_id', 
	'Fuel Type': 'ft',
    'Source Type_Fuel Type Label': 'sutft_lab', 
	'sort_order_pol': 'sort_order_pol', 
	'MOVES3 Pollutant ID': 'pollutant_id',
	'Short Name MOVES3': 'short_name_mvs3', 
	'EIS Pollutant Code': "eis_polcd", 
	'NEI Pollutant Name': 'short_name_nei17',
	'excluded_pollutant_from_xml': 'excluded_pollutant_from_xml', 
	'Process ID': 'process_id', 
	'Process Label': 'prc_lab',
    'Process Name': 'prc_nm', 
	'MOVES3 SCC': 'mvs_scc', 
	'NEI SCC': 'nei_scc', 
	'txled_fac': 'txled_fac',
    'Emission (Tons)': 'emis_short_tons',  
	'Activity Type ID': 'actty_id',
    'Activity Type Abb': 'acttyabb', 
	'Activity Type': 'acttylab', 
	'activity': 'act', 
	'Units': 'act_unit'
}

## Transform Data


### Get District County Map

In [92]:
fips_254 = list(district_map.FIPS_ST_CN.unique())
analysis_fips = fips_254
district_map_anly = district_map.loc[
    district_map.FIPS_ST_CN.isin(analysis_fips)
].rename(columns={"FIPS_ST_CN": "fips", "CNTY_NM": "cnty_nm", "TXDOT_DIST": "txdot_dist"})

df_district = df_district.rename(columns={"DIST_NM": "district", "DIST_NBR": "txdot_dist"})
district_map_anly = df_district.merge(district_map_anly, on="txdot_dist")
district_map_anly.loc[lambda df: df.district == "Fort Worth"]

,district,txdot_dist,fips,cnty_nm
113,Fort Worth,2,48221,Hood
114,Fort Worth,2,48251,Johnson
115,Fort Worth,2,48363,Palo Pinto
116,Fort Worth,2,48367,Parker
117,Fort Worth,2,48425,Somervell
118,Fort Worth,2,48143,Erath
119,Fort Worth,2,48497,Wise
120,Fort Worth,2,48439,Tarrant
121,Fort Worth,2,48237,Jack


### Process HPMS VMT-Mix


In [93]:
vmtmix_hpms.groupby(["Year", "RoadType", "DayType", "TimeType", "District"]).agg(
    vmt_hpms=("Fraction-HPMS", "sum")
).describe()


,vmt_hpms
count,1.500000e+02
mean,1.000000e+00
std,3.310740e-16
min,1.000000e+00
25%,1.000000e+00
50%,1.000000e+00
75%,1.000000e+00
max,1.000000e+00


In [94]:
vmtmix_hpms.groupby(["Year", "RoadType", "DayType", "TimeType", "District"]).agg(
    vmt_fy23=("Fraction", "sum")
).describe()


,vmt_fy23
count,1.500000e+02
mean,1.000000e+00
std,4.283285e-08
min,9.999999e-01
25%,1.000000e+00
50%,1.000000e+00
75%,1.000000e+00
max,1.000000e+00


In [95]:
vmtmix_hpms = vmtmix_hpms.rename(columns={"Vehicle type": "hpmsvehtype"})
vmtmix_hpms.hpmsvehtype.unique()
map_vcat = {
    "Motorcycles": 10,
    "Passenger Cars": 251,
    "Light truck": 252,
    "Bus": 40,
    "ST": 50,
    "CT": 60,
}
vmtmix_hpms["hpms_idx"] = vmtmix_hpms.hpmsvehtype.map(map_vcat)


In [96]:
map_sut = {
    11: 10,
    21: 251,
    31: 252,
    32: 252,
    41: 40,
    42: 40,
    43: 40,
    51: 50,
    52: 50,
    53: 50,
    54: 50,
    61: 60,
    62: 60,
}
vmtmix_filt["hpms_idx"] = vmtmix_filt.source_type_id.map(map_sut)


In [97]:
vmtmix_hpms = vmtmix_hpms.rename(
    columns={
        "Vehicle type": "hpmsvehtype",
        "Year": "yearID",
        "RoadType": "hpms_rdtype",
        "DayType": "dowagg",
        "TimeType": "tod",
        "District": "district",
        "Fraction-HPMS": "vmtmix_hpms_agg",
        "Fraction": "vmtmix_axb_agg",
        "Difference (TTI-HPMS)": "vmtmix_axb_hpms_agg_diff",
        "% of Difference": "vmtmix_axb_hpms_agg_perdiff",
    }
)
vmtmix_hpms


,yearID,hpms_rdtype,hpmsvehtype,dowagg,tod,district,vmtmix_hpms_agg,vmtmix_axb_agg,vmtmix_axb_hpms_agg_diff,vmtmix_axb_hpms_agg_perdiff,hpms_idx
36475,2020,1,Motorcycles,Wkd,day,Abilene,0.0010,0.002172,0.001172,1.172154,10
36476,2020,1,Motorcycles,Wkd,day,Amarillo,0.0010,0.003576,0.002576,2.576365,10
36477,2020,1,Motorcycles,Wkd,day,Atlanta,0.0010,0.001709,0.000709,0.709073,10
36478,2020,1,Motorcycles,Wkd,day,Austin,0.0010,0.001950,0.000950,0.950011,10
36479,2020,1,Motorcycles,Wkd,day,Beaumont,0.0010,0.001395,0.000395,0.395230,10
...,...,...,...,...,...,...,...,...,...,...,...
53995,2020,6,CT,Wkd,day,San Antonio,0.0629,0.085213,0.022313,0.354732,60
53996,2020,6,CT,Wkd,day,Tyler,0.0629,0.095128,0.032228,0.512375,60
53997,2020,6,CT,Wkd,day,Waco,0.0629,0.122263,0.059363,0.943767,60
53998,2020,6,CT,Wkd,day,Wichita Falls,0.0629,0.201943,0.139043,2.210539,60


In [98]:
map_hpms_idx_lab = {
    1: "Urban Interstate",
    2: "Rural Interstate",
    3: "Urban Other Arterials",
    4: "Urban Others",
    5: "Rural Other Arterials",
    6: "Rural Others",
}
map_hpms_mvs = {1: "u_ra", 2: "r_ra", 3: "u_ura", 4: "u_ura", 5: "r_ura", 6: "r_ura"}
mvs_lab_idx = {"r_ra": 2, "r_ura": 3, "u_ra": 4, "u_ura": 5}
map_hpms_mvs_idx = {key: mvs_lab_idx[val] for key, val in map_hpms_mvs.items()}
map_hpms_mvs_idx


{1: 4, 2: 2, 3: 5, 4: 5, 5: 3, 6: 3}

In [99]:
vmtmix_filt

,dgcode,txdot_dist,district,mvs_rdtype_nm,mvs_rdtype,dowagg,yearID,tod,sourceTypeName,source_type_id,fuel_type_id,fuelTypeDesc,vmt_mix_fy23,hpms_idx
14304,W,8,Abilene,r_ra,2,Wkd,2020,day,Motorcycle,11,1,Gasoline,0.001633,10
14305,W,8,Abilene,r_ra,2,Wkd,2020,day,Passenger Car,21,1,Gasoline,0.362703,251
14306,W,8,Abilene,r_ra,2,Wkd,2020,day,Passenger Car,21,2,Diesel Fuel,0.002571,251
14307,W,8,Abilene,r_ra,2,Wkd,2020,day,Passenger Truck,31,1,Gasoline,0.205808,252
14308,W,8,Abilene,r_ra,2,Wkd,2020,day,Passenger Truck,31,2,Diesel Fuel,0.008775,252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820795,G,13,Yoakum,u_ura,5,Wkd,2020,day,Motor Home,54,1,Gasoline,0.001421,50
820796,G,13,Yoakum,u_ura,5,Wkd,2020,day,Motor Home,54,2,Diesel Fuel,0.000780,50
820797,G,13,Yoakum,u_ura,5,Wkd,2020,day,Combination Short-haul Truck,61,1,Gasoline,0.000017,60
820798,G,13,Yoakum,u_ura,5,Wkd,2020,day,Combination Short-haul Truck,61,2,Diesel Fuel,0.127789,60


In [100]:
vmtmix_filt_1 = vmtmix_filt.filter(
    items=[
        "district",
        "mvs_rdtype",
        "source_type_id",
        "fuel_type_id",
        "hpms_idx",
        "vmt_mix_fy23",
    ]
).assign(
    vmt_mix_agg=lambda df: df.groupby(
        ["district", "mvs_rdtype", "hpms_idx"]
    ).vmt_mix_fy23.transform(sum),
    frac_sut_in_hpms=lambda df: df.vmt_mix_fy23 / df.vmt_mix_agg
).drop(columns="vmt_mix_fy23")


In [101]:
vmtmix_hpms["mvs_rdtype"] = vmtmix_hpms.hpms_rdtype.map(map_hpms_mvs_idx).astype(str)
vmtmix_hpms["hpms_rdtype_nm"] = vmtmix_hpms.hpms_rdtype.map(map_hpms_idx_lab)
vmtmix_hpms_1 = vmtmix_hpms.merge(vmtmix_filt_1, on=["district", "mvs_rdtype", "hpms_idx"])
vmtmix_hpms_1["vmt_mix_hpms_sut"] =  vmtmix_hpms_1.vmtmix_hpms_agg * vmtmix_hpms_1.frac_sut_in_hpms

In [102]:
vmtmix_hpms_2 = vmtmix_hpms_1.filter(items=["district", "yearID", "dowagg", "tod", "hpms_rdtype", "hpms_rdtype_nm", "mvs_rdtype", "source_type_id", "fuel_type_id", "vmt_mix_hpms_sut"])

In [103]:
assert set(district_map_anly.district.unique()).symmetric_difference( set(vmtmix_hpms_2.district.unique())) == set()
vmtmix_hpms_cnty = vmtmix_hpms_2.merge(district_map_anly, on="district")
vmtmix_hpms_cnty

,district,yearID,dowagg,tod,hpms_rdtype,hpms_rdtype_nm,mvs_rdtype,source_type_id,fuel_type_id,vmt_mix_hpms_sut,txdot_dist,fips,cnty_nm
0,Abilene,2020,Wkd,day,1,Urban Interstate,4,11,1,0.001000,8,48417,Shackelford
1,Abilene,2020,Wkd,day,1,Urban Interstate,4,11,1,0.001000,8,48253,Jones
2,Abilene,2020,Wkd,day,1,Urban Interstate,4,11,1,0.001000,8,48151,Fisher
3,Abilene,2020,Wkd,day,1,Urban Interstate,4,11,1,0.001000,8,48033,Borden
4,Abilene,2020,Wkd,day,1,Urban Interstate,4,11,1,0.001000,8,48415,Scurry
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36571,Yoakum,2020,Wkd,day,6,Rural Others,3,62,2,0.033189,13,48057,Calhoun
36572,Yoakum,2020,Wkd,day,6,Rural Others,3,62,2,0.033189,13,48469,Victoria
36573,Yoakum,2020,Wkd,day,6,Rural Others,3,62,2,0.033189,13,48321,Matagorda
36574,Yoakum,2020,Wkd,day,6,Rural Others,3,62,2,0.033189,13,48239,Jackson


In [104]:
vmtmix_hpms_cnty_1 = vmtmix_hpms_cnty.filter(
    items=["hpms_rdtype", "hpms_rdtype_nm","fips", "cnty_nm", "district", "mvs_rdtype", "source_type_id", "fuel_type_id", "vmt_mix_hpms_sut"]
)
vmtmix_hpms_cnty_1

,hpms_rdtype,hpms_rdtype_nm,fips,cnty_nm,district,mvs_rdtype,source_type_id,fuel_type_id,vmt_mix_hpms_sut
0,1,Urban Interstate,48417,Shackelford,Abilene,4,11,1,0.001000
1,1,Urban Interstate,48253,Jones,Abilene,4,11,1,0.001000
2,1,Urban Interstate,48151,Fisher,Abilene,4,11,1,0.001000
3,1,Urban Interstate,48033,Borden,Abilene,4,11,1,0.001000
4,1,Urban Interstate,48415,Scurry,Abilene,4,11,1,0.001000
...,...,...,...,...,...,...,...,...,...
36571,6,Rural Others,48057,Calhoun,Yoakum,3,62,2,0.033189
36572,6,Rural Others,48469,Victoria,Yoakum,3,62,2,0.033189
36573,6,Rural Others,48321,Matagorda,Yoakum,3,62,2,0.033189
36574,6,Rural Others,48239,Jackson,Yoakum,3,62,2,0.033189


In [105]:
vmtmix_hpms_cnty_2 = vmtmix_hpms_cnty_1.loc[lambda df: df.hpms_rdtype.isin([1,2,3,5])]

In [106]:
vmtmix_hpms_pvttab = vmtmix_hpms_1.pivot_table(index=["hpms_idx", "hpms_rdtype", "hpmsvehtype"], columns=["source_type_id", "fuel_type_id"], values=["vmtmix_hpms_agg", "frac_sut_in_hpms"])
vmtmix_hpms_pvttab.to_csv(r"C:\Users\a-bibeka\Downloads\vmt_hpms_pivot_table.csv")

In [107]:
assert np.allclose(vmtmix_hpms_1.vmtmix_axb_agg, vmtmix_hpms_1.vmt_mix_agg, atol=1e6)


### Process FY23 VMT-Mix


In [108]:
vmtmix_filt_anly = vmtmix_filt.merge(district_map_anly, on=["txdot_dist", "district"], how="outer")
vmtmix_filt_anly.isna().sum()

dgcode            0
txdot_dist        0
district          0
mvs_rdtype_nm     0
mvs_rdtype        0
dowagg            0
yearID            0
tod               0
sourceTypeName    0
source_type_id    0
fuel_type_id      0
fuelTypeDesc      0
vmt_mix_fy23      0
hpms_idx          0
fips              0
cnty_nm           0
dtype: int64

In [109]:
vmtmix_filt_anly_1 = vmtmix_filt_anly.filter(
    items=["fips", "cnty_nm", "district", "mvs_rdtype", "source_type_id", "fuel_type_id", "vmt_mix_fy23"]
)
vmtmix_filt_anly_1

,fips,cnty_nm,district,mvs_rdtype,source_type_id,fuel_type_id,vmt_mix_fy23
0,48417,Shackelford,Abilene,2,11,1,0.001633
1,48253,Jones,Abilene,2,11,1,0.001633
2,48151,Fisher,Abilene,2,11,1,0.001633
3,48033,Borden,Abilene,2,11,1,0.001633
4,48415,Scurry,Abilene,2,11,1,0.001633
...,...,...,...,...,...,...,...
24379,48057,Calhoun,Yoakum,5,62,2,0.042679
24380,48469,Victoria,Yoakum,5,62,2,0.042679
24381,48321,Matagorda,Yoakum,5,62,2,0.042679
24382,48239,Jackson,Yoakum,5,62,2,0.042679


### Get Pollutants of Interest


In [121]:
suft_emis = suft_emis.rename(columns=rename_sutft_emis)
suft_emis1 = suft_emis.loc[lambda df: df.period == "annual"]

In [110]:

rate_map_filt = rate_map.loc[lambda df: df.pollutantName.isin(POLS)]
pol_filt = (
    rate_map_filt[["pollutantID", "pollutantName"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
pol_emis = (
    suft_emis[["pollutant_id", "short_name_mvs3", "short_name_nei17"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
pol_filt_1 = pol_filt.merge(pol_emis, left_on="pollutantID", right_on="pollutant_id")

### Process Trends 2020 VMT


In [122]:
sutft_act = sutft_act.rename(columns=rename_sutft_emis)
sutft_act1 = sutft_act.loc[lambda df: df.period == "annual"]

In [123]:
sutft_vmt = sutft_act1.loc[sutft_act.acttyabb == "vmt"]
sutft_vmt = sutft_vmt.rename(columns={"act": "vmt"})
sutft_vmt["vmt_agg"] = sutft_vmt.groupby(
    [
        "cnty_nm",
        "fips",
        "year_id",
        "rdtype",
        "mvs_rdtype",
        "actty_id",
        "acttyabb",
        "acttylab",
    ]
).vmt.transform(sum)

sutft_vmt = sutft_vmt.filter(
    items=[
        "cnty_nm",
        "fips",
        "rdtype",
        "mvs_rdtype",
        "actty_id",
        "acttyabb",
        "acttylab",
        "source_type_id",
        "fuel_type_id",
        "vmt",
        "vmt_agg",
    ]
)

### Merge Emission to VMT


In [124]:

suft_emis_vmt = suft_emis1.merge(
    sutft_vmt,
    on=[
        "cnty_nm",
        "fips",
        "mvs_rdtype",
        "rdtype",
        "source_type_id",
        "fuel_type_id",
    ],
    how="left",
)
suft_emis_vmt_onroad = suft_emis_vmt.loc[
    lambda df: (df.mvs_rdtype != 1) & (df.vmt != 0)
]

suft_emis_vmt_onroad["emission_rate_calc"] = (
    suft_emis_vmt_onroad.emis_short_tons / suft_emis_vmt_onroad.vmt
)

suft_emis_vmt_onroad_anly = suft_emis_vmt_onroad.loc[
    lambda df: df.fips.isin(analysis_fips)
]


C:\Users\a-bibeka\AppData\Local\Temp\ipykernel_19460\1500677135.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suft_emis_vmt_onroad["emission_rate_calc"] = (


In [156]:
test = suft_emis_vmt_onroad_anly.merge(vmtmix_filt_anly_1, on=["fips", "mvs_rdtype", "source_type_id", "fuel_type_id"], how="outer", suffixes=["_trends", "_fy23"])

In [170]:
test1 = test.loc[lambda df: (df.short_name_mvs3 == "CO") & (df.process_id == 1)].groupby(["fips", "mvs_rdtype", "short_name_mvs3", "process_id"]).vmt_mix_fy23.sum()

In [165]:
test1 = test.loc[lambda df: (df.short_name_mvs3 == "CO") & (df.process_id == 1)].groupby(["mvs_rdtype"]).fips.nunique()

In [149]:
grps_trends = suft_emis_vmt_onroad_anly.loc[lambda df: (df.short_name_mvs3 == "CO") & (df.process_id == 1)].filter(items=["fips", "year_id", "mvs_rdtype", "source_type_id", "fuel_type_id", "short_name_mvs3", "process_id"]).reset_index(drop=True).assign(source="trends")
vmtmix_filt_anly_1 = vmtmix_filt_anly_1.assign(source="fy23_iac", mvs_rdtype=lambda df: df.mvs_rdtype.astype(int))

,period,year_id,fips,cnty_nm,mvs_rdtype,rdtype,source_type_id,sut,fuel_type_id,ft,...,mvs_scc,nei_scc,txled_fac,emis_short_tons,actty_id,acttyabb,acttylab,vmt,vmt_agg,emission_rate_calc
1467,annual,2020,48001,Anderson,3,Rural Unrestricted Access,11,Motorcycle,1,Gasoline,...,2201110301,2201110080,1.0,10.513284,1.0,vmt,Vehicle Miles of Travel,712168.431229,3.217848e+08,1.476236e-05
1468,annual,2020,48001,Anderson,3,Rural Unrestricted Access,11,Motorcycle,1,Gasoline,...,2201110315,2201110080,1.0,0.000000,1.0,vmt,Vehicle Miles of Travel,712168.431229,3.217848e+08,0.000000e+00
1469,annual,2020,48001,Anderson,3,Rural Unrestricted Access,11,Motorcycle,1,Gasoline,...,2201110301,2201110080,1.0,0.519341,1.0,vmt,Vehicle Miles of Travel,712168.431229,3.217848e+08,7.292385e-07
1470,annual,2020,48001,Anderson,3,Rural Unrestricted Access,11,Motorcycle,1,Gasoline,...,2201110315,2201110080,1.0,0.000000,1.0,vmt,Vehicle Miles of Travel,712168.431229,3.217848e+08,0.000000e+00
1471,annual,2020,48001,Anderson,3,Rural Unrestricted Access,11,Motorcycle,1,Gasoline,...,2201110301,2201110080,1.0,0.085630,1.0,vmt,Vehicle Miles of Travel,712168.431229,3.217848e+08,1.202386e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891799,annual,2020,48505,Zapata,5,Urban Unrestricted Access,62,Combination Long-haul Truck,2,Diesel Fuel,...,2202620509,2202620080,1.0,0.005682,1.0,vmt,Vehicle Miles of Travel,194766.815356,3.350904e+07,2.917387e-08
891800,annual,2020,48505,Zapata,5,Urban Unrestricted Access,62,Combination Long-haul Truck,2,Diesel Fuel,...,2202620510,2202620080,1.0,0.001118,1.0,vmt,Vehicle Miles of Travel,194766.815356,3.350904e+07,5.737759e-09
891801,annual,2020,48505,Zapata,5,Urban Unrestricted Access,62,Combination Long-haul Truck,2,Diesel Fuel,...,2202620515,2202620080,1.0,0.004961,1.0,vmt,Vehicle Miles of Travel,194766.815356,3.350904e+07,2.547010e-08
891802,annual,2020,48505,Zapata,5,Urban Unrestricted Access,62,Combination Long-haul Truck,2,Diesel Fuel,...,2202620501,2202620080,1.0,0.067403,1.0,vmt,Vehicle Miles of Travel,194766.815356,3.350904e+07,3.460700e-07


In [143]:
suft_emis_vmt_onroad_anly.columns

Index(['period', 'year_id', 'fips', 'cnty_nm', 'mvs_rdtype', 'rdtype',
       'source_type_id', 'sut', 'fuel_type_id', 'ft', 'sutft_lab',
       'sort_order_pol', 'pollutant_id', 'short_name_mvs3', 'eis_polcd',
       'short_name_nei17', 'excluded_pollutant_from_xml', 'process_id',
       'prc_lab', 'prc_nm', 'mvs_scc', 'nei_scc', 'txled_fac',
       'emis_short_tons', 'actty_id', 'acttyabb', 'acttylab', 'vmt', 'vmt_agg',
       'emission_rate_calc'],
      dtype='object')

In [144]:
vmtmix_filt_anly_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24384 entries, 0 to 24383
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   fips            24384 non-null  int32  
 1   cnty_nm         24384 non-null  object 
 2   district        24384 non-null  object 
 3   mvs_rdtype      24384 non-null  int32  
 4   source_type_id  24384 non-null  int64  
 5   fuel_type_id    24384 non-null  int64  
 6   vmt_mix_fy23    24384 non-null  float64
 7   source          24384 non-null  object 
dtypes: float64(1), int32(2), int64(2), object(3)
memory usage: 1.5+ MB


In [145]:
grps_trends.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12980 entries, 0 to 12979
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   fips            12980 non-null  int64 
 1   year_id         12980 non-null  int64 
 2   mvs_rdtype      12980 non-null  int64 
 3   source_type_id  12980 non-null  int64 
 4   fuel_type_id    12980 non-null  int64 
 5   source          12980 non-null  object
dtypes: int64(5), object(1)
memory usage: 608.6+ KB


In [147]:
missing_grps = vmtmix_filt_anly_1.merge(grps_trends, on=["fips", "mvs_rdtype", "source_type_id", "fuel_type_id"], how="outer", suffixes=["_fy23", "_trends"])

In [148]:
missing_grps

,fips,cnty_nm,district,mvs_rdtype,source_type_id,fuel_type_id,vmt_mix_fy23,source_fy23,year_id,source_trends
0,48417,Shackelford,Abilene,2,11,1,0.001633,fy23_iac,NaN,NaN
1,48253,Jones,Abilene,2,11,1,0.001633,fy23_iac,2020.0,trends
2,48151,Fisher,Abilene,2,11,1,0.001633,fy23_iac,NaN,NaN
3,48033,Borden,Abilene,2,11,1,0.001633,fy23_iac,NaN,NaN
4,48415,Scurry,Abilene,2,11,1,0.001633,fy23_iac,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
24379,48057,Calhoun,Yoakum,5,62,2,0.042679,fy23_iac,2020.0,trends
24380,48469,Victoria,Yoakum,5,62,2,0.042679,fy23_iac,2020.0,trends
24381,48321,Matagorda,Yoakum,5,62,2,0.042679,fy23_iac,2020.0,trends
24382,48239,Jackson,Yoakum,5,62,2,0.042679,fy23_iac,2020.0,trends


In [130]:
debug = missing_grps.loc[missing_grps.source_trends.isna()]

In [131]:
debug.columns

Index(['fips', 'cnty_nm', 'district', 'mvs_rdtype', 'source_type_id',
       'fuel_type_id', 'vmt_mix_fy23', 'source_fy23', 'year_id',
       'source_trends'],
      dtype='object')

In [132]:
debug.fips.nunique()

254

In [133]:
debug.district.nunique()

25

In [134]:
debug.mvs_rdtype.nunique()

4

In [135]:
debug.mvs_rdtype.nunique()

4